In [1]:
import pandas as pd
import time
import urllib.request, urllib.parse, json
import pickle

import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# 4/5wEbCysPwm4FPLQ9C_53agEElQ74SbY4cu2Zr3PXPFW-GLpjfsVyuAY

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/My Drive/Capstone_ML_Guild/Capstone/Data_Collection/Groundwater_APY_Data_Step1.csv')

In [4]:
df.isna().sum()

State                       0
District                    0
GW_depth_Minimum            0
GW_depth_Maximum            0
Wells_depth_0_to_2          0
Wells_depth_2_to_5          0
Wells_depth_5_to_10         0
Wells_depth_10_to_20        0
Wells_depth_20_to_40        0
Wells_depth_40_and_above    0
Crop_Year                   0
Season                      0
Crop                        0
Area                        0
Production                  0
dtype: int64

In [5]:
state_district = df.groupby(['State','District']).any().index.values

In [6]:
geo_data = {}
requested_index = 0

# read geo data from pkl file
try:
  geo_file = open('/content/drive/My Drive/Capstone_ML_Guild/Capstone/Data_Collection/geo_data.pkl', 'rb')
  geo_data = pickle.load(geo_file)                     
  geo_file.close()
  print(f"No. of geo locations found - {len(geo_data)}")
except:
  print('Fetch geolocation data from server.')

No. of geo locations found - 30


In [7]:
if len(geo_data) == 0:
  def get_data():
    global requested_index
    global state_district
    global geo_data
    if requested_index != len(state_district):
      try:
        for i in range(requested_index, len(state_district)):
          state, district = state_district[i]
          q = urllib.parse.quote(f"{district},{state},India")
          req_url = f"http://geocode.xyz/{q}?json=1"
          with urllib.request.urlopen(req_url) as url:
            data = json.loads(url.read().decode())
            if(state not in geo_data.keys()):
              geo_data[state] = {}
            geo_data[state][district] = {"long": data['longt'], "lat": data['latt']}
            print(i, state, district, geo_data[state][district])
            requested_index += 1
      except:
        time.sleep(5)
        get_data()

  get_data()

  # write geo data to pkl file
  out_file = open('/content/drive/My Drive/Capstone_ML_Guild/Capstone/Data_Collection/geo_data.pkl', 'ab')
  pickle.dump(geo_data, out_file)
  out_file.close()


In [8]:
df_with_geodata = df.copy()

In [9]:
lat_long = []
for state, district in df_with_geodata[['State', 'District']].values:
  lat_long.append([geo_data[state][district]['lat'], geo_data[state][district]['long']])

In [10]:
df_lat_long = pd.DataFrame(lat_long, columns=['Lat', 'Long'])
df_with_geodata = pd.concat((df_with_geodata, df_lat_long), axis = 1)

In [11]:
df_with_geodata.tail()

,State,District,GW_depth_Minimum,GW_depth_Maximum,Wells_depth_0_to_2,Wells_depth_2_to_5,Wells_depth_5_to_10,Wells_depth_10_to_20,Wells_depth_20_to_40,Wells_depth_40_and_above,Crop_Year,Season,Crop,Area,Production,Lat,Long
214466,WEST BENGAL,WEST MEDINIPUR,0.7,16.9,5,8,10,13.0,0.0,0.0,2014,Summer,Rice,195960.0,580442.0,22.42410,87.32094
214467,WEST BENGAL,WEST MEDINIPUR,0.7,16.9,5,8,10,13.0,0.0,0.0,2014,Summer,Sesamum,74080.0,58514.0,22.42410,87.32094
214468,WEST BENGAL,WEST MEDINIPUR,0.7,16.9,5,8,10,13.0,0.0,0.0,2014,Whole Year,Sugarcane,4390.0,789614.0,22.42410,87.32094
214469,WEST BENGAL,WEST MEDINIPUR,0.7,16.9,5,8,10,13.0,0.0,0.0,2014,Winter,Rice,496134.0,1348329.0,22.42410,87.32094
214470,WEST BENGAL,WEST MEDINIPUR,0.7,16.9,5,8,10,13.0,0.0,0.0,2014,Winter,Sesamum,119.0,79.0,22.42410,87.32094


In [12]:
df_with_geodata.columns

Index(['State', 'District', 'GW_depth_Minimum', 'GW_depth_Maximum',
       'Wells_depth_0_to_2', 'Wells_depth_2_to_5', 'Wells_depth_5_to_10',
       'Wells_depth_10_to_20', 'Wells_depth_20_to_40',
       'Wells_depth_40_and_above', 'Crop_Year', 'Season', 'Crop', 'Area',
       'Production', 'Lat', 'Long'],
      dtype='object')

In [13]:
rearranged_cols = ['State', 'District', 'Lat', 'Long', 'GW_depth_Minimum', 'GW_depth_Maximum',
       'Wells_depth_0_to_2', 'Wells_depth_2_to_5', 'Wells_depth_5_to_10',
       'Wells_depth_10_to_20', 'Wells_depth_20_to_40',
       'Wells_depth_40_and_above', 'Crop_Year', 'Season', 'Crop', 'Area',
       'Production']
df_with_geodata = df_with_geodata[rearranged_cols]
df_with_geodata.head()

,State,District,Lat,Long,GW_depth_Minimum,GW_depth_Maximum,Wells_depth_0_to_2,Wells_depth_2_to_5,Wells_depth_5_to_10,Wells_depth_10_to_20,Wells_depth_20_to_40,Wells_depth_40_and_above,Crop_Year,Season,Crop,Area,Production
0,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Arecanut,3100.0,5200.0
1,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Other Kharif pulses,668.0,448.0
2,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Rice,10779.0,31863.0
3,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Whole Year,Banana,1531.0,12073.0
4,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Whole Year,Cashewnut,80.0,54.0


In [14]:
df_with_geodata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214471 entries, 0 to 214470
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   State                     214471 non-null  object 
 1   District                  214471 non-null  object 
 2   Lat                       214471 non-null  object 
 3   Long                      214471 non-null  object 
 4   GW_depth_Minimum          214471 non-null  float64
 5   GW_depth_Maximum          214471 non-null  float64
 6   Wells_depth_0_to_2        214471 non-null  int64  
 7   Wells_depth_2_to_5        214471 non-null  int64  
 8   Wells_depth_5_to_10       214471 non-null  int64  
 9   Wells_depth_10_to_20      214471 non-null  float64
 10  Wells_depth_20_to_40      214471 non-null  float64
 11  Wells_depth_40_and_above  214471 non-null  float64
 12  Crop_Year                 214471 non-null  int64  
 13  Season                    214471 non-null  o

In [15]:
df_with_geodata['Lat'] = df_with_geodata['Lat'].astype(float)
df_with_geodata['Long'] = df_with_geodata['Long'].astype(float)

In [16]:
(df_with_geodata.Lat.min(), df_with_geodata.Long.min())

(8.08783, 69.611)

In [17]:
(df_with_geodata.Lat.max(), df_with_geodata.Long.max())

(33.22028, 96.65348)

In [18]:
df_with_geodata.to_csv('/content/drive/My Drive/Capstone_ML_Guild/Capstone/Data_Collection/Geo_GW_APY_Data_Step2.csv', index=False)